In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import torch.nn.functional as F

In [5]:
class Model1(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.fc1 = nn.Linear(20, 30)
        self.fc2 = nn.Linear(30, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
class Model2(nn.Module):
    def __init__(self, num_classes=5):
        super(Model2, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.dropout = nn.Dropout2d(p=0.2)  # Added dropout layer
        self.pool = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 114 * 114, 5)  # Updated input size

    def forward(self, input):
        input = input.float()
        output = F.relu(self.bn1(self.conv1(input)))
        output = F.relu(self.bn2(self.conv2(output)))
        output = self.pool(output)
        output = self.dropout(output)  # Applied dropout
        output = F.relu(self.bn4(self.conv4(output)))
        output = F.relu(self.bn5(self.conv5(output)))
        #print(output.size())
        output = output.view(-1, 64 * 114 * 114)  # Adjusted view size
        output = self.fc1(output)
        return output

# Create ensemble class
class Ensemble(nn.Module):
    def __init__(self, models):
        super(Ensemble, self).__init__()
        self.models = nn.ModuleList(models)

    def forward(self, x):
        outputs = [model(x) for model in self.models]
        avg_output = torch.mean(torch.stack(outputs), dim=0)
        return avg_output


In [6]:

# Generate synthetic data
X = torch.randn(1000, 20)
y = torch.randint(0, 2, (1000,))

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create DataLoader for training and validation sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32)

# Initialize base models
model1 = Model1()
model2 = Model2()

# Train base models
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)

for epoch in range(10):
    model1.train()
    for inputs, labels in train_loader:
        optimizer1.zero_grad()
        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer1.step()

    model2.train()
    for inputs, labels in train_loader:
        optimizer2.zero_grad()
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer2.step()

# Create ensemble
ensemble = Ensemble([model1, model2])

# Evaluate ensemble on validation set
ensemble.eval()
val_preds = []
val_labels = []
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = ensemble(inputs)
        _, preds = torch.max(outputs, 1)
        val_preds.extend(preds.numpy())
        val_labels.extend(labels.numpy())

val_accuracy = accuracy_score(val_labels, val_preds)
print("Validation Accuracy:", val_accuracy)


C:\Users\murra\AppData\Local\Temp\ipykernel_23444\2927216099.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
C:\Users\murra\AppData\Local\Temp\ipykernel_23444\2927216099.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.long)
C:\Users\murra\AppData\Local\Temp\ipykernel_23444\2927216099.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
C:\Users\murra\AppData\Local\T

TypeError: super(type, obj): obj must be an instance or subtype of type